In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('lending_club_ml.csv')
df.drop(['id'], inplace=True, axis=1)

In [4]:
# creating columns for features that had more than 2 outcomes
# I have decided to use subgrade and not grade as these are suppose to be similar features but, subgrade is more granular

df = pd.concat([df, pd.get_dummies(
    df.sub_grade, prefix='sub_grade', drop_first=True)], axis=1)
df = pd.concat([df, pd.get_dummies(df.home_ownership,
                                   prefix='home_ownership', drop_first=True)], axis=1)
df = pd.concat([df, pd.get_dummies(df.verification_status,
                                   prefix='verification_status', drop_first=True)], axis=1)
df = pd.concat(
    [df, pd.get_dummies(df.purpose, prefix='purpose', drop_first=True)], axis=1)
#df = pd.concat([df, pd.get_dummies(df.verification_status_joint,
#                                   prefix='verification_status_joint', drop_first=True)], axis=1)

df.drop(columns=['sub_grade', 'home_ownership', 'verification_status',
                 'purpose', 'verification_status_joint'], inplace=True)


# conversions for features that only had 2 outcomes

df.disbursement_method = df.disbursement_method.apply(
    lambda disburstment: 1 if disburstment == 'Cash' else 0)

df.application_type = df.application_type.apply(
    lambda app_type: 1 if app_type == 'Individual' else 0)

In [5]:
#fill all nans with 0
df.fillna(0, inplace=True)

In [6]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score


def scoring(clf, x, y):
    
    #Baseline
    print('Loan passing rate:', np.mean(y))
    print('Balanced loan passing rate:', np.average(y, weights=x['loan_amnt']))
    print('\n')
    
    
    print('score: ', clf.score(x, y))
    # score adjusted for loan amount
    print('balanced_accuracy_score: ', balanced_accuracy_score(
        y, clf.predict(x), sample_weight=x['loan_amnt']))

    print(confusion_matrix(y, clf.predict(x)))
    
    print('F1 score: ', f1_score(y, clf.predict(x)))
    
    print('precision_score: ', precision_score(y, clf.predict(x)))

    # score adjusted for loan amount
    print('average_precision_score: ', average_precision_score(
        y, clf.predict(x), average='weighted', sample_weight=x['loan_amnt']))
    
    print('recall_score: ', recall_score(y, clf.predict(x)))

    print('roc: ', roc_auc_score(y, clf.predict_proba(x)[:, 1]))

    # score adjusted for loan amount
    print('roc_weighted: ', roc_auc_score(y, clf.predict_proba(x)[
          :, 1], average='weighted', sample_weight=x['loan_amnt']))

In [7]:
#random forest on base data

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X = df.drop(['grade','loan_status'], axis=1)
y = df.loan_status

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_best = RandomForestClassifier(criterion= 'entropy', n_estimators= 101)
rf_best.fit(X_train, y_train)
    
print('RANDOM FOREST')
scoring(rf_best, X_test, y_test)

RANDOM FOREST
Loan passing rate: 0.8009672589365736
Balanced loan passing rate: 0.7855872316459166


score:  0.9413408118379516
balanced_accuracy_score:  0.8595251067935064
[[107850  41827]
 [  2286 600059]]
F1 score:  0.9645459725726171
precision_score:  0.934837338717467
average_precision_score:  0.9289095297501627
recall_score:  0.9962048327785571
roc:  0.9645804810670238
roc_weighted:  0.9654295899880078


For only single loans

In [8]:
df_single = df.loc[df.application_type == 1]

joint_list = ['sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il',
            'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog', 'dti_joint', 'revol_bal_joint', 'annual_inc_joint']

df_single.drop(joint_list,axis=1,inplace=True)

In [9]:
Xs = df_single.drop(['grade','loan_status'], axis=1)
ys = df_single.loan_status

Xs_train, Xs_test, ys_train, ys_test = train_test_split(Xs, ys, test_size=0.3, random_state=42)

rf_s = RandomForestClassifier(criterion= 'entropy', n_estimators= 101)
rf_s.fit(Xs_train, ys_train)
    
print('RANDOM FOREST')
scoring(rf_s, Xs_test, ys_test)

RANDOM FOREST
Loan passing rate: 0.8018762389397847
Balanced loan passing rate: 0.7866714294905874


score:  0.9412825194283301
balanced_accuracy_score:  0.8594455359135631
[[105634  41187]
 [  2326 591910]]
F1 score:  0.9645467041137166
precision_score:  0.934943618434458
average_precision_score:  0.9293009612109121
recall_score:  0.9960857302485882
roc:  0.9639677189747058
roc_weighted:  0.9648368529952163


For the core features, features that I believe many investors just consider.

In [10]:
df_modded = df[['loan_status','loan_amnt','annual_inc','installment','int_rate','fico_range_high']]

df_modded['installment_to_annual_inc'] =  df_modded.annual_inc/df_modded.installment * 12
df_modded.drop(['annual_inc','installment'],axis=1,inplace=True)

In [11]:
Xm = df_modded.drop(['loan_status'], axis=1)
ym = df_modded.loan_status

Xm_train, Xm_test, ym_train, ym_test = train_test_split(Xm, ym, test_size=0.3, random_state=42)

rf_m = RandomForestClassifier(criterion= 'entropy', n_estimators= 101)
rf_m.fit(Xm_train, ym_train)
    
print('RANDOM FOREST')
scoring(rf_m, Xm_test, ym_test)

RANDOM FOREST
Loan passing rate: 0.8009672589365736
Balanced loan passing rate: 0.7855872316459166


score:  0.9088284651246905
balanced_accuracy_score:  0.8276352162690431
[[101705  47972]
 [ 20591 581754]]
F1 score:  0.9443514213060774
precision_score:  0.9238208363637519
average_precision_score:  0.9148445502581968
recall_score:  0.9658152719786833
roc:  0.9134875855259501
roc_weighted:  0.9160065258876731
